In [1]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin

import numpy as np


In [2]:
# load David's 2-BTP model
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG/chem_annotated.inp'
RMG_dictionary_path = 'models/RMG/species_dictionary.txt'
RMG_transport_path = 'models/RMG/tran.dat'
RMG_cti_path = 'models/RMG/chem_annotated.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
# RMG_gas = ct.Solution(RMG_cti_path)



In [3]:
# generate a list of species with halogens, then remove those from the model reactions
halogens = []
for sp in RMG_species_list:
    for atom in sp.molecule[0].atoms:
        if atom.is_halogen():
            halogens.append(sp)
            break
# generate a list of species with halogens, then remove those from the model reactions
halogen_reactions = []
for rxn in RMG_reaction_list:
    for reactant in rxn.reactants:
        if reactant in halogens:
            halogen_reactions.append(rxn)
            break

In [4]:
truncated_sp_list = []
for sp in RMG_species_list:
    if sp not in halogens:
        truncated_sp_list.append(sp)

truncated_rxn_list = []
for rxn in RMG_reaction_list:
    if rxn not in halogen_reactions:
        truncated_rxn_list.append(rxn)
        
print(len(truncated_sp_list))
print(len(truncated_rxn_list))


98
825


In [5]:
rmgpy.chemkin.save_chemkin_file('no_halogens.inp', truncated_sp_list, truncated_rxn_list, verbose=True, check_for_duplicates=True)
rmgpy.chemkin.save_transport_file('no_halogens_tran.dat', truncated_sp_list)
rmgpy.chemkin.save_species_dictionary('no_halogens_dict.txt', truncated_sp_list)